In [1]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

In [3]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 35
#ram_dimnetion = 9 if partial
nb_actions = 25
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

FileNotFoundError: [Errno 2] No such file or directory: './logs/dssat-pdi.log'

In [4]:
Action_list=[]
Nleach_list = []
Yield_list=[]
reward_list=[]
rain_list=[]
for i in range(1):
    #print('')
    rain=0
    date=0
    v6_date = 0
    v20_date = 0
    v7_date = 0
    env.reset()
    rain=rain+env.observation['rain']
    W_input = 0
    #print(env.observation)
    #date = 1
    #print(date)
    c1=1;
    c2=1;
    c3=1;
    c4=1;
    c5=1;
    n_list=[]
    w_list=[]
    istage_list=[]
    vstage_list=[]
    while not env.done:
        #print(observation)
        #print(observation)
        # observation_list = env.observation_dict_to_array(observation)
        w_input=0
        n_input=0
        date = date+1
        if env.observation['vstage']<6:
            if date %10==1:
                w_input = 12.7
                print('water 1')
        if env.observation['vstage'] == 6:
            if c4==1:
                v6_date = date
                c4=c4-1
        if env.observation['vstage'] >= 6:
            if env.observation['vstage']<20:
                if (date-v6_date)%7 ==0:
                    w_input=12.7
                    print('water 2')
        if env.observation['vstage'] == 20:
            if c5==1:
                c5=c5-1
                v20_date = date
        if env.observation['vstage'] >= 20:
            if env.observation['istage'] < 6:
                if (date-v20_date)%3==0:
                    w_input=12.7
                    print('water 3')
        if env.observation['istage'] == 7:
            if c1 ==1:
                n_input = 40
                c1=c1-1
                print('fert 1 = 40')
        if env.observation['vstage'] == 5:
            if c2 ==1:
                n_input=40
                c2=c2-1
                print('fert 2 = 40')
        if env.observation['vstage'] ==7:
            if c3 ==1:
                n_input =150
                c3=c3-1
                print('fert 3 = 150')
                v7_date = date
        if env.observation['vstage'] >= 7:
            if date-v7_date == 28:
                n_input= 130
                print('fert 4 = 130')
        action = {
                    'anfer': n_input,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha4
                    'amir': w_input,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
        #print(date)
        n_list.append(action['anfer'])
        w_list.append(action['amir'])
        istage_list.append(env.observation['istage'])
        vstage_list.append(env.observation['vstage'])
        rain=rain+env.observation['rain']
        observation, reward, done, info = env.step(action)
        #print(observation['yrdoy'])
        #print('')
        #print(observation)
        #real_ob = add_fert(observation, action_dict['anfer'])
        #print('real_ob is ', real_ob)
        #env.observation = real_ob
        #print('final ob is:', env.observation)
        #real_s=get_real_state(env._state, action_dict['anfer'])
        #env._state=real_s
        #print('env_state', env._state)
        #print(date)
        #print(reward)
        #print(done)
        if done:
            final_ob=env.observation
            #print(final_ob)
            N_input=final_ob['cumsumfert']
            Nleach=final_ob['cleach']
            Yield=final_ob['grnwt']
            Action_list.append(N_input)
            Nleach_list.append(Nleach)
            Yield_list.append(Yield)
            reward=Yield*0.158-0.79*N_input-1.1*W_input
            reward_list.append(reward)
            rain_list.append(rain)
            #print(env.history['action'])
            #print(env.history['reward'])
#print(np.mean(rain_list))
print('W amount is', w_input)
print('N amount is', Action_list)
#print('avergae N input is', np.mean(Action_list))
#print('')
print('N leaching is', Nleach_list)
#print('avergae Leaching is', np.mean(Nleach_list))
#print('')
print('Yield is', Yield_list)
#print('avergae Yield is', np.mean(Yield_list))
#print('')
print('Reward is', reward_list)
#print('avergae reward is', np.mean(reward_list))
import pandas
df = pandas.DataFrame(n_list, columns=["colummn"])
df.to_csv('n_list.csv', index=False)
df = pandas.DataFrame(w_list, columns=["colummn"])
df.to_csv('w_list.csv', index=False)
df = pandas.DataFrame(istage_list, columns=["colummn"])
df.to_csv('istage_list.csv', index=False)
df = pandas.DataFrame(vstage_list, columns=["colummn"])
df.to_csv('vstage_list.csv', index=False)

NameError: name 'env' is not defined

In [4]:
env.reset()
print(env.observation)

{'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}


In [30]:
Nleach_list

[10.648062705993652,
 15.5799560546875,
 12.338714599609375,
 2.379901170730591,
 22.064546585083008,
 5.136173725128174,
 32.15522766113281,
 25.89251708984375,
 1.1620451211929321,
 14.372591018676758]

In [8]:
Yield_list

[9600.956420898438,
 4562.802429199219,
 5943.7042236328125,
 9623.003540039062,
 13368.8037109375,
 12950.1904296875,
 9539.637451171875,
 13334.02099609375,
 4795.851135253906,
 10534.593505859375]

In [9]:
reward_list

[942.2311254501343,
 440.038681936264,
 572.0489852905273,
 945.7836225986481,
 1316.0961780548096,
 1276.2494220733643,
 937.7455790758133,
 1314.9191820144654,
 457.27873306274415,
 1036.522788619995]

In [ ]:
b=a[2]

In [9]:
i=1

In [11]:
str(i)+'happy.csv'

'1happy.csv'

In [4]:
penalty = -200

In [5]:
penalty = max(penalty,-100)

In [6]:
penalty

-100